In [1]:
import os
import numpy as np
import pandas as pd
import re

In [2]:
#samplelist
samplelist=["Patient1","Patient2","Patient3"]
files=["amp","del"]
Peaks=["Amplification Peak","Deletion Peak"]

In [3]:
os.chdir("/Users/sarunach/Desktop/Gistic/")

**Driver cancer genes**

In [4]:
#Step1 : read the driver genes specific for the tumor typr
df1=pd.read_excel("drivers.xlsx",index_col=None)

**Gistic results**

In [5]:
#Step2 : read lesions file, amp file, del file. Add cancer driver genes to  amp and del file into excel .
for sample in samplelist:    
    for file,Peak in zip(files,Peaks):  
        print(file,Peak)
        BASE_DIR="/Users/sarunach/Desktop/Gistic/gistic_input/"# first mount cluster  on local computer to work directly with cluster
        os.chdir(BASE_DIR+sample+"_results")
        df2=pd.read_csv(file+"_genes.conf_75.txt", header=[0,3],delimiter="\t")## skip rows 2,3,4
        df2=df2.drop(df2.columns[0],axis=1) #drop first column
        df2.columns = df2.columns.map('_'.join) # join the headers
        df2 = df2.loc[:, ~df2.columns.str.contains('^Unnamed')] # remove any unnamed columns
        colnum = len(df2.columns)
        #removing the square brackets around genes
        for col in df2.columns:
            df2[col] = df2[col].str.replace('[','').str.replace(']','') # replace any [] in dataframe
        #reading all lesions file
        band=pd.read_csv("all_lesions.conf_75.txt",delimiter="\t",skipinitialspace=True)
        band=band.filter(regex="Unique|Descriptor|Wide|Sample") # chosing columns of interest
        band['Descriptor'] = band['Descriptor'].str.strip()
        band['Wide Peak Limits'] = band['Wide Peak Limits'].str.strip()
        band["id"]=band["Descriptor"]+ "_"+ band["Wide Peak Limits"] # joining two columns and creating new one
        band["id"]=band.id.str.split("(").apply(lambda x: x[0]) # spliting column on "(" and keeping only first part
        band=band[(band["Unique Name"].str.contains(Peak) == True) & (band["Unique Name"].str.contains("values") == False)]
        rowlen=len(band)
        print(colnum==rowlen)
    
        collist=df2.columns
        if colnum==rowlen:
            df4 = pd.DataFrame()
            for col in collist:
                df3=df2[df2[col].isin(df1.Gene)] # finding driver genes
                df3=df3.filter([col])
                df4=df4.append(df3, ignore_index=True,sort=False)
        df5=df4.T #transpose the dataframe for merging 
        df5.index.name='id' # name the index
        df5 = df5.reset_index(drop=False) # resetting the column names 
        df5["Cancer_driver_gene"]=df5[df5.columns[1:]].apply( lambda x: ",".join(x.dropna().astype(str)),axis=1)
        df5=df5.filter(["id","Cancer_driver_gene"])
        final=pd.merge(band,df5, on="id")
        os.chdir("/Users/sarunach/Desktop/Gistic/gistic_output/")
        if not os.path.exists(sample+"_Processed_Output"):
            os.mkdir(sample+"_Processed_Output")
        os.chdir("/Users/sarunach/Desktop/Gistic/gistic_output/" + sample+"_Processed_Output")
        final.to_excel(sample+"_"+file+"_focal_driver_combined.xlsx" , index=None)

amp Amplification Peak
True
del Deletion Peak
True
amp Amplification Peak
True
del Deletion Peak
True
amp Amplification Peak
True
del Deletion Peak
True


In [6]:
#Step3: combine amp and del file into excel 
def combine_excel():
    os.chdir("/Users/sarunach/Desktop/Gistic/gistic_output/" + sample+"_Processed_Output")
    keyword="combined"
    all_file=pd.DataFrame()
    for filename in os.listdir(): 
        if keyword in filename:
            df = pd.read_excel(filename)
            all_file=all_file.append(df)
    all_file.to_excel(sample+"_"+"amp_del"+"_focal_driver_combined.xlsx" , index=None)

In [7]:
for sample in samplelist:
    combine_excel()

**##END##**